In [1]:
import argparse
import logging
import os
import pprint
from logging import getLogger

In [2]:
import numpy as np

In [39]:
import requests

In [3]:
from ape import Contract, accounts, chain, networks

C:\Users\chuck.raghavan\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import find_dotenv, load_dotenv

In [5]:
from giza.agents import AgentResult, GizaAgent


In [6]:
from addresses import ADDRESSES

In [7]:
# from lp_tools import get_tick_range
# from uni_helpers import (approve_token, check_allowance, close_position,
#                          get_all_user_positions, get_mint_params)

In [8]:
load_dotenv(find_dotenv())

True

In [9]:
dev_passphrase = os.environ.get("DEV_PASSPHRASE")
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")
GIZA1_PASSPHRASE = os.environ.get("GIZA1_PASSPHRASE")

In [10]:
logging.basicConfig(level=logging.INFO)

In [11]:
def create_agent(
    model_id: int, version_id: int, chain: str, contracts: dict, account: str
):
    """
    Create a Giza agent for the regression model
    """
    agent = GizaAgent(
        contracts=contracts,
        id=model_id,
        version_id=version_id,
        chain=chain,
        account=account,
    )
    return agent

In [12]:
def predict(agent: GizaAgent, X: np.ndarray):
    """
    Predict the next day volatility.

    Args:
        X (np.ndarray): Input to the model.

    Returns:
        int: Predicted value.
    """
    prediction = agent.predict(input_feed={"val": X}, verifiable=True, job_size="XL")
    return prediction

In [13]:
def get_pred_val(prediction: AgentResult):
    """
    Get the value from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: Predicted value.
    """
    # This will block the executon until the prediction has generated the proof
    # and the proof has been verified
    return prediction.value[0][0]

In [14]:
# def eval_quote(
#     #agent_id: int,
#     pred_model_id:int,
#     pred_version_id:int,
#     policyId:int,
#     quote: float,
#     bmi:float,
#     account="chainaimlabs3003",
#     chain=f"ethereum:sepolia:{sepolia_rpc_url}",
# ):
    
#     ## Change the INS-ENROLLMENT-AGENT_PASSPHRASE to be {AGENT-NAME}_PASSPHRASE
#     # os.environ["INS_ENROLLMENT-AGENT_PASSPHRASE"] = os.environ.get("DEV_PASSPHRASE")

#     # Create logger

#     logger = getLogger("agent_logger")

#     networks.parse_network_choice(f"ethereum:sepolia:{sepolia_rpc_url}").__enter__()
#     chain_id = chain.chain_id

#     # Load the addresses
  
#     insuranceEnrollmentContractAddress = Contract(ADDRESSES["INSE"][chain_id])

#     #wallet_address = accounts.load(account).address

#     # Load the data, this can be changed to retrieve live data
#     #file_path = "data/data_array.npy"
#     #X = np.load(file_path)
     
#     #input = np.array([[30.1]]).astype(np.float32)

#     # input = np.array([[bmi]]).astype(np.float32)
#     inputTotalETF = 453.1;

#     input = np.array([[inputTotalETF]]).astype(np.float32)

#     # Fill this contracts dictionary with the contract addresses that our agent will interact with
#     contracts = {
#         "insuranceEnrollmentContract": insuranceEnrollmentContractAddress,
#     }

#     # Create the agent

#     agent = create_agent(
#         model_id=pred_model_id,
#         version_id=pred_version_id,
#         chain=chain,
#         contracts=contracts,
#         account=account,
#     )

#     result = predict(agent, input)

#     #if (quote<result){

#     #if(true)(

#     with agent.execute() as contracts:
        
#         logger.info("Executing INSE contract")
#         contracts.InsuranceEnrollmentContract.buyPolicy(policyId)
#         logger.info("Executing INSE contract")
#     #)

#     """
#     Get the value from the prediction.

#     Args:
#         prediction (dict): Prediction from the model.

#     Returns:
#         int: 

#     """
#     # This will block the executon until the prediction has generated the proof
#     # and the proof has been verified

#     print(result)

#     return result

In [40]:
def get_eth_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    return data['ethereum']['usd']

In [41]:
current_eth_price = get_eth_price()
current_eth_price

3678.04

In [31]:
def  rebalance_lp(
    tokenWETH_amount: int,
    tokenUSDC_amount: int,
    pred_model_id: int,
    pred_version_id: int,
    account="giza1",
    chain1=f"ethereum:sepolia:{sepolia_rpc_url}",
    nft_id=None,
):
    logger = getLogger("agent_logger")
    networks.parse_network_choice(f"ethereum:sepolia:{sepolia_rpc_url}").__enter__()

    chain_id = chain.chain_id
    # weth_mint_amount = 0.01
    weth_mint_amount = tokenWETH_amount
    pool_fee = 3000
    uni = Contract(ADDRESSES["UNI"][chain_id])
    weth = Contract(ADDRESSES["WETH"][chain_id])
    # wbtc = Contract(ADDRESSES["WETH"][chain_id])
    wusdc = Contract(ADDRESSES["USDC"][chain_id])
    print("weth address: ", weth)
    # wbtc = Contract('0x66194f6c999b28965e0303a84cb8b797273b6b8b')
    weth_decimals = weth.decimals()
    # wbtc_decimals = wbtc.decimals()
    uni_decimals = uni.decimals()
    wusdc_decimals = wusdc.decimals()
    weth_mint_amount = int(weth_mint_amount * 10**weth_decimals)
    uni_mint_amount = int(0.5 * weth_mint_amount)
    contracts = {
        "weth": weth,
        "wusdc": wusdc,
    }
    # nft_manager_address = ADDRESSES["NonfungiblePositionManager"][11155111]
    # tokenA_address = ADDRESSES["UNI"][11155111]
    # tokenB_address = ADDRESSES["WETH"][11155111]
    # pool_address = "0x287B0e934ed0439E2a7b1d5F0FC25eA2c24b64f7"
    # print("pool address: ", pool_address)
    # contracts = {
    #     "nft_manager": nft_manager_address,
    #     "tokenA": tokenA_address,
    #     "tokenB": tokenB_address,
    #     "pool": pool_address,
    # }    


    inputTotalETF = 453.1;
    input = np.array([[inputTotalETF]]).astype(np.float32)

    # Create the agent

    agent = create_agent(
        model_id=pred_model_id,
        version_id=pred_version_id,
        chain=chain1,
        contracts=contracts,
        account=account,
    )

    result = predict(agent, input)
    print("model result = ", result)

    with agent.execute() as contracts:
    
        logger.info("Executing contract")
        # contracts.InsuranceEnrollmentContract.buyPolicy(policyId)
        # logger.info("Executing INSE contract")
        # print(f"Your WETH balance: {contracts.weth.balanceOf(wallet.address)/10**weth_decimals}")
        # print(f"Your WUSDC balance: {contracts.wusdc.balanceOf(wallet.address)/10**wusdc_decimals}")
    
    # with accounts.use_sender("giza1"):
    #     # print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    #     # weth.deposit(value=weth_mint_amount)
    #     # print("Approving WETH for swap")
    #     # weth.approve(swap_router.address, weth_mint_amount)
    #     swap_params = {
    #         "tokenIn": weth.address,
    #         "tokenOut": wusdc.address,
    #         "fee": pool_fee,
    #         "recipient": wallet.address,
    #         "amountIn": weth_mint_amount,
    #         "amountOutMinimum": 0,
    #         "sqrtPriceLimitX96": 0,
    #     }
    #     swap_params = tuple(swap_params.values())
    #     print("Swapping WETH for USDC")
    #     # amountOut = swap_router.exactInputSingle(swap_params)
    #     # print(f"Successfully minted {uni_mint_amount/10**uni_decimals} USDC")
    

    

In [37]:
def  rebalance_lp_NOMODEL(
    tokenWETH_amount: int,
    tokenUSDC_amount: int,
    pred_model_id: int,
    pred_version_id: int,
    account="giza1",
    chain1=f"ethereum:sepolia:{sepolia_rpc_url}",
    nft_id=None,
):
    logger = getLogger("agent_logger")
    networks.parse_network_choice(f"ethereum:sepolia:{sepolia_rpc_url}").__enter__()

    chain_id = chain.chain_id
    # weth_mint_amount = 0.01
    weth_mint_amount = tokenWETH_amount
    pool_fee = 3000
    uni = Contract(ADDRESSES["UNI"][chain_id])
    weth = Contract(ADDRESSES["WETH"][chain_id])
    # wbtc = Contract(ADDRESSES["WETH"][chain_id])
    wusdc = Contract(ADDRESSES["USDC"][chain_id])
    print("weth address: ", weth)
    # wbtc = Contract('0x66194f6c999b28965e0303a84cb8b797273b6b8b')
    weth_decimals = weth.decimals()
    # wbtc_decimals = wbtc.decimals()
    uni_decimals = uni.decimals()
    wusdc_decimals = wusdc.decimals()
    weth_mint_amount = int(weth_mint_amount * 10**weth_decimals)
    uni_mint_amount = int(0.5 * weth_mint_amount)
  
    wallet = accounts.load("giza1")

    # inputTotalETF = 453.1;
    # input = np.array([[inputTotalETF]]).astype(np.float32)

    # Create the agent

    # agent = create_agent(
    #     model_id=pred_model_id,
    #     version_id=pred_version_id,
    #     chain=chain1,
    #     contracts=contracts,
    #     account=account,
    # )

    # result = predict(agent, input)
    # print("model result = ", result)

    # with agent.execute() as contracts:
    
    logger.info("Executing contract")
    print(f"Your WETH balance: {weth.balanceOf(wallet.address)/10**weth_decimals}")
    print(f"Your WUSDC balance: {wusdc.balanceOf(wallet.address)/10**wusdc_decimals}")

In [33]:
MODEL_ID = 739
VERSION_ID = 1

print(MODEL_ID)
print(VERSION_ID)

tokenWETH_amount = 2500000
tokenUSDC_amount = 1000000

rebalance_lp(tokenWETH_amount, tokenUSDC_amount, MODEL_ID, VERSION_ID)

739
1
weth address:  0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14


INFO:giza.agents.model:Could not download model: 'GizaAgent' object has no attribute '_output_path'
INFO:giza.agents.model:Model already downloaded at: C:\Users\CHUCK~1.RAG\AppData\Local\Temp\739_1_linear_regression-betf6 ✅
INFO:giza.agents.model:Predicting
ERROR:giza.agents.model:An error occurred in predict: 500 Server Error: Internal Server Error for url: https://endpoint-giza1-739-1-a8a598d9-7i3yxzspbq-ew.a.run.app/cairo_run
ERROR:giza.agents.model:Deployment predict error: Error processing proof for request
ERROR:giza.agents.model:An error occurred in predict: 500 Server Error: Internal Server Error for url: https://endpoint-giza1-739-1-a8a598d9-7i3yxzspbq-ew.a.run.app/cairo_run


HTTPError: 500 Server Error: Internal Server Error for url: https://endpoint-giza1-739-1-a8a598d9-7i3yxzspbq-ew.a.run.app/cairo_run

### No model call - just Ape contracts

In [38]:
MODEL_ID = 739
VERSION_ID = 1

print(MODEL_ID)
print(VERSION_ID)

tokenWETH_amount = 2500000
tokenUSDC_amount = 1000000

rebalance_lp_NOMODEL(tokenWETH_amount, tokenUSDC_amount, MODEL_ID, VERSION_ID)

739
1
weth address:  0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14


INFO:agent_logger:Executing contract


Your WETH balance: 0.02
Your WUSDC balance: 5.436197


In [26]:
if __name__ == "__main__":
     
    # Create the parser

    # parser = argparse.ArgumentParser()
    # # # Add arguments
    # parser.add_argument("--model-id", metavar="M", type=int, help="model-id")
    # parser.add_argument("--version-id", metavar="V", type=int, help="version-id")
    # parser.add_argument("--quote", metavar="A", type=int, help="quote")
    # # parser.add_argument("--tokenB-amount", metavar="B", type=int, help="tokenB-amount")
    # # # Parse arguments
    # args=parser.parse_args()

    # for now we will just get it from .env
    #model_id = os.environ.get("MODEL_ID")
    #version_id = os.environ.get("VERSION_ID")
    
    MODEL_ID = 739
    VERSION_ID = 1

    print(MODEL_ID)
    print(VERSION_ID)

    tokenWETH_amount = 2500000
    tokenUSDC_amount = 1000000

    rebalance_lp(tokenWETH_amount, tokenUSDC_amount, MODEL_ID, VERSION_ID)

739
1
weth address:  0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14


INFO:giza.agents.model:Could not download model: 'GizaAgent' object has no attribute '_output_path'
INFO:giza.agents.model:Model already downloaded at: C:\Users\CHUCK~1.RAG\AppData\Local\Temp\739_1_linear_regression-betf6 ✅
INFO:giza.agents.model:Predicting
INFO:giza.agents.model:Serialized: [1 1] [3933011291 false]


🚀 Starting deserialization process...
✅ Deserialization completed! 🎉


INFO:giza.agents.agent:Updating agent with chain ethereum:sepolia:https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI
INFO:giza.agents.agent:Updating agent with latest contracts


model result =  AgentResult(input={'val': array([[453.1]], dtype=float32)}, request_id=7525e11b94024d8c820b15f04816d4be, value=[[60012.9896698]])


TypeError: Object of type ContractInstance is not JSON serializable